## Canvas Learning chatbot

Welcome! This is ... Important things:
- Every line of code here is processed either on the machine you're using to host this notebook.
- you may choose to export any of the data from this notebook wherever you choose
- Chatbot logs are not persisted (in a file sense), and will disappear when you stop the notebook
    - Stretch goal: make them persistable?
    
The goal of putting this in a notebook is to ensure that the handling of your data is as transparent as possible. Please read through the entirety of the notebook before consenting to give this notebook your token. Ask questions! Make noise! Submit issues!

In [39]:
import getpass

print("Do you understand that your access token will give this app")
print("permission to use/delete any of your info on Canvas? (yes to continue)")

consent = input()

while consent != "yes":
    consent = input()

token = getpass.getpass("Enter your access token here: ")

Do you understand that your access token will give this app
permission to use/delete any of your info on Canvas? (yes to continue)
yes
Enter your access token here: ········


In [99]:
# Convenience methods

def api(path):
    '''Returns the Canvas API url for the slash-prefixed path'''
    BASE_URL = "https://canvas.ubc.ca"
    BASE_PATH = "/api/v1"
    return BASE_URL + BASE_PATH + path

def r_get(**kwargs):
    '''Convenience method to grab stuff
    Input { key1: param1, key2: param2 }
    Output: data from api(/key1/param1/key2/param2)'''
    path = "/"
    params = {}
    
    for name, value in kwargs.items():
        if name == "params":
            params = value
            continue
            
        path += str(name) + "/"
        if not value:
            break
        else:
            path += str(value) + "/"
            
    print("Requesting value at <%s>" % api(path))
    return requests.get(api(path), headers=headers, params=params)

In [135]:
# Setup requests. Pull course data

import requests

headers = {"Authorization": "Bearer " + token}

r = requests.get(api("/courses"), headers=headers)

courses = r.json()

In [136]:
print("Which course would you like to get help for? ([xx])")
i = 0
scanned_courses = []
for c in courses:
    if "name" in c.keys():
        print("[%s] %s" % (i, c["name"]))
        scanned_courses.append(c)
        i += 1

index = int(input())
course = scanned_courses[index]
print(course['name'] + " it is!")

Which course would you like to get help for? ([xx])
[0] APSC 450 101 Professional Engineering Practice
[1] Canvas API Hackathon 🤖
[2] CIVL 250 99A Engineering and Sustainable Development
[3] CPSC 259 101 Data Structures and Algorithms for Electrical Engineers
[4] MECH 326 Mechanical Design II
4
MECH 326 Mechanical Design II it is!


In [157]:
# Index the material in the course
r = requests.get(api("/courses/" + str(course['id']) + "/modules"), headers=headers)
modules = r.json()

# List of all item urls
module_item_urls = [m['items_url'] for m in modules]

# Items in each module
modules_items = [requests.get(url, headers=headers).json() for url in module_item_urls]

In [162]:
modules_htmls = []
for module_items in modules_items:
    module_htmls = [module_item['html_url'] for module_item in module_items if 'html_url' in module_item.keys()]
    modules_htmls.append(module_htmls)

module_htmls

['https://canvas.ubc.ca/courses/14709/modules/items/846154',
 'https://canvas.ubc.ca/courses/14709/modules/items/860937',
 'https://canvas.ubc.ca/courses/14709/modules/items/846153']

In [166]:
r = requests.get(module_htmls[0], headers=headers)
r

<Response [200]>

In [167]:
r.text

'<!-- UBCIT Communication -->\n<!DOCTYPE html>\n<html>\n        <head>\n\t\t<meta http-equiv="Content-Type" content="text/html; charset=UTF-8">\n        <meta charset="utf-8">\n        <meta name="viewport" content="width=device-width, initial-scale=1.0">\n\t\t<meta name="description">\n\t\t<meta name="author">\n\t\t<title>The University of British Columbia</title>\n        <link rel="stylesheet" type="text/css" href="/idp/css/main.css">\n        <link rel="stylesheet" type="text/css" href="/idp/css/ubc-clf-full.css">\n        <link rel="stylesheet" type="text/css" href="/idp/css/unit.css">\t\n\t\t<!--[if lte IE 7]>\n    \t<link href="/idp/css/font-awesome-ie7.css" rel="stylesheet" />\n\t\t<![endif]-->\n\t\t<!--[if lt IE 9]>\n    \t  <script src="/idp/scripts/html5.js"></script>\n\t\t<![endif]-->\n\t\t<!-- Fav and touch icons -->\n\t\t<link rel="shortcut icon" href="/idp/images/favicon.ico">\n\t\t<link href="/idp/images/apple-touch-icon-144-precomposed.png" sizes="144x144">\n\t\t<link 

In [163]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_doc, 'html.parser')


NameError: name 'html_doc' is not defined

In [118]:
module_ids = [m['id'] for m in modules]

r = requests.get(api("/courses/%s/modules/%s" % 
                    (str(course['id']), str(module_ids[0]))),
                headers=headers)

In [126]:
# r1 = r_get(courses=course['id'], 
#            modules=module_ids[0], 
#            items=590803, 
#            params={})
# r1.json()
items = r_get(courses=course['id'],
             modules=module_ids[0],
             items=None).json()
item_ids = [i['id'] for i in items]

r2 = [r_get(courses=course['id'], 
           modules=module_ids[0], 
           items=i).json() for i in item_ids]

#requests.get(r.json()['items_url'], headers=headers).json()

Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/768167/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/768106/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/774277/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/789236/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/789292/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/768168/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/774447/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/774448/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996/items/774449/>
Requesting value at <https://canvas.ubc.ca/api/v1/courses/14709/modules/81996

AttributeError: 'list' object has no attribute 'json'

In [127]:
r2

[{'id': 768167,
  'title': 'Pre-tutorial videos',
  'position': 1,
  'indent': 0,
  'type': 'SubHeader',
  'module_id': 81996},
 {'id': 768106,
  'title': 'Pre-tutorial Video 0-1',
  'position': 2,
  'indent': 0,
  'type': 'Assignment',
  'module_id': 81996,
  'html_url': 'https://canvas.ubc.ca/courses/14709/modules/items/768106',
  'content_id': 220340,
  'url': 'https://canvas.ubc.ca/api/v1/courses/14709/assignments/220340'},
 {'id': 774277,
  'title': 'Pre-tutorial Video 0-2',
  'position': 3,
  'indent': 0,
  'type': 'Assignment',
  'module_id': 81996,
  'html_url': 'https://canvas.ubc.ca/courses/14709/modules/items/774277',
  'content_id': 223784,
  'url': 'https://canvas.ubc.ca/api/v1/courses/14709/assignments/223784'},
 {'id': 789236,
  'title': 'Pre-tutorial Video 0-3',
  'position': 4,
  'indent': 0,
  'type': 'Assignment',
  'module_id': 81996,
  'html_url': 'https://canvas.ubc.ca/courses/14709/modules/items/789236',
  'content_id': 227632,
  'url': 'https://canvas.ubc.ca/api